Matthew Virgin  
COS 575  
Homework 3  
5 April 2024  

Load the data

In [1]:
import numpy as np
import scipy.io as sio

bodyfat = sio.loadmat("bodyfat_data.mat")
X = bodyfat['X']
Y = bodyfat['y']
n,d = X.shape

Split the data

In [2]:
## get 150 training samples, rest are test

split_loc = 150

x_train = X[:split_loc]
y_train = Y[:split_loc]

x_test = X[split_loc:]
y_test = Y[split_loc:]

Define nescessary functions

In [3]:
## relu (activation function of choice)
def relu(x):
    return np.maximum(0, x)

## derivative of relu
def d_relu(x):
    return np.where(x > 0, 1, 0)
    
## mean squared error (loss function)
## ytr represents y_train
def MSE(y_hat, ytr):
    return np.mean((y_hat-ytr)**2)

## forward prop
def forward_pass(X,W1,W2,W3,b1,b2,b3):
    z1=np.dot(X,W1)+b1
    a1=relu(z1)
    z2=np.dot(a1,W2)+b2
    a2=relu(z2)
    y_hat =np.dot(a2,W3)+b3
    return z1,z2,y_hat,a1,a2

## backward prop
def backward_pass(Xtr,ytr,learning_rate,y_hat,W1,W2,W3,b1,b2,b3,z1,z2,a1,a2):
    #loss
    print(MSE(y_hat, ytr))
    #backpropogation code
    l3_error=(y_hat-ytr)/float(ytr.shape[0])
    l2_error=l3_error.dot(W3.T)
    l2_delta=l2_error*d_relu(z2)
    l1_error=l2_delta.dot(W2.T)
    l1_delta=l1_error*d_relu(z1)
    #update parameters using GD
    W3-= learning_rate*a2.T.dot(l3_error)
    W2-= learning_rate*a1.T.dot(l2_delta)
    W1-= learning_rate*Xtr.T.dot(l1_delta)
    b3-= learning_rate*np.sum(l3_error)
    b2-= learning_rate*np.sum(l2_delta,axis = 0)
    b1-= learning_rate*np.sum(l1_delta,axis = 0)
    return W1,W2,W3,b1,b2,b3

## training
def train(Xtr,ytr,learning_rate,epochs,W1,W2,W3,b1,b2,b3): 
    for i in range(epochs):
        z1,z2,y_hat,a1,a2 = forward_pass(Xtr,W1,W2,W3,b1,b2,b3)
        W1,W2,W3,b1,b2,b3 = backward_pass(Xtr,ytr,learning_rate,y_hat,W1,W2,W3,b1,b2,b3,z1,z2,a1,a2)
    return W1,W2,W3,b1,b2,b3

## take final weights from training, make prediction on set x, and evaluate
## against ground truth y
def pred(x,y,W1,W2,W3,b1,b2,b3):
    z1,z2,y_hat,a1,a2 = forward_pass(x, W1,W2,W3,b1,b2,b3)
    return MSE(y_hat, y)

Put it all together & train

In [4]:
np.random.seed(0)

## initialize parameters
learning_rate = 1e-7
l1_size = 64
l2_size = 16
w1 = np.random.randn(d, l1_size)  
b1 = np.zeros((1, l1_size))
w2 = np.random.randn(l1_size, l2_size)
b2 = np.zeros((1, l2_size))
w3 = np.random.randn(l2_size, 1)    # output size is 1
b3 = np.zeros((1, 1))

## train for 10,000 epochs on train set, store final weights
w1,w2,w3,b1,b2,b3 = train(x_train,y_train,learning_rate,10000,w1,w2,w3,b1,b2,b3)

17365121.638620447
157390.52591884558
45669.0807749324
18467.874503544717
11468.083735907594
9599.510083918123
9060.621509293562
8880.79580045364
8794.617135030869
8732.552370484975
8672.317876732455
8609.955326186699
8543.826436819882
8469.719633888317
8366.654133159298
8294.200184675417
8260.702333163194
8226.245229890577
8193.228485129965
8161.447958127774
8130.048147791143
8099.211378715268
8068.845324095483
8038.671183243756
8008.339997675074
7977.771038006153
7947.454321153656
7917.440821652659
7885.797474085195
7854.779126691935
7823.692544495557
7790.004652618009
7756.647634730672
7723.511973576591
7690.506250396731
7657.708391583121
7624.9397545064585
7592.102291381753
7558.5559833462285
7523.395436640504
7487.842549481044
7451.968327347404
7415.850719671279
7377.635846232341
7339.209185142691
7299.986872480425
7257.441652989467
7212.772967970983
7167.505953288622
7117.806802184572
7063.398038697956
7004.927592824533
6938.349630663306
6867.258613856261
6785.90885373252
6688.18

Get final evaluations on train & test sets

In [5]:
## evaluate on train
print("Train MSE is", pred(x_train, y_train, w1,w2,w3,b1,b2,b3))

## evaluate on test
print("Test MSE is", pred(x_test, y_test, w1,w2,w3,b1,b2,b3))

Train MSE is 21.426124131738476
Test MSE is 25.545574510021808
